# Run Test Method

In [8]:
def runTests(testClass):
    loader = unittest.TestLoader()
    suite = loader.loadTestsFromTestCase(testClass)
    runner = unittest.TextTestRunner()
    runner.run(suite)

# Modelo

In [9]:
from __future__ import annotations
import ast, sys
from dataclasses import dataclass
from typing import List, Iterable, Tuple, Optional

@dataclass(frozen=True)
class CodeWarning:
    message: str
    lineno: int
    col: int
    end_lineno: Optional[int] = None
    end_col: Optional[int] = None

class DetectionRule:
    name: str = "Generic Detector"
    def search(self, src: str) -> List[CodeWarning]:
        raise NotImplementedError

class Patcher:
    name: str = "Generic Patcher"
    def patch(self, tree: ast.Node) -> str:
        raise NotImplementedError

class DCCLinter:
    def __init__(self, detectors: Iterable[DetectionRule], patchers: Iterable[Patcher]):
        self.detectors = detectors
        self.patchers = patchers

    def search(self, src: str) -> List[CodeWarning]:
        out: List[CodeWarning] = []
        for d in self.detectors:
            out.extend(d.search(src))
        return sorted(out, key=lambda w: (w.lineno, w.col))

    def patch(self, src: str) -> str:
        tree = ast.parse(src)
        for p in self.patchers:
          tree = p.patch(tree)
        return ast.unparse(tree)

# None Compares

## Detector

In [10]:
class OpenCloseDetectionRule(DetectionRule):
    name = "Detect open(...); ...; f.close()"

    def search(self, src: str) -> List[CodeWarning]:
        # Detecta patrón open/uso/close secuencial.
        tree = ast.parse(src)
        warnings: List[CodeWarning] = []
        for i, stmt in enumerate(tree.body):
            if isinstance(stmt, ast.Assign) and isinstance(stmt.value, ast.Call) and isinstance(stmt.value.func, ast.Name) and stmt.value.func.id == 'open':
                if i + 2 < len(tree.body):
                    end = tree.body[i + 2]
                    if (isinstance(end, ast.Expr) and isinstance(end.value, ast.Call) and isinstance(end.value.func, ast.Attribute)
                        and end.value.func.attr == 'close' and isinstance(end.value.func.value, ast.Name)
                        and isinstance(stmt.targets[0], ast.Name) and end.value.func.value.id == stmt.targets[0].id):
                        warnings.append(CodeWarning("Usa 'with open(...) as f:' en lugar de open/close manual", stmt.lineno, stmt.col_offset))
        return warnings

## Patcher

In [11]:
class OpenClosePatcher(Patcher):
    name = "Open/Close → with open as f"

    def patch(self, tree: ast.AST) -> ast.AST:
        # Reescribe secuencia open/uso/close a with.
        new_body, i = [], 0
        while i < len(tree.body):
            stmt = tree.body[i]
            if (isinstance(stmt, ast.Assign) and isinstance(stmt.value, ast.Call)
                and isinstance(stmt.value.func, ast.Name) and stmt.value.func.id == 'open'
                and i + 2 < len(tree.body)):
                mid, end = tree.body[i + 1], tree.body[i + 2]
                if (isinstance(end, ast.Expr) and isinstance(end.value, ast.Call) and isinstance(end.value.func, ast.Attribute)
                    and end.value.func.attr == 'close' and isinstance(end.value.func.value, ast.Name)
                    and isinstance(stmt.targets[0], ast.Name) and end.value.func.value.id == stmt.targets[0].id):
                    var = stmt.targets[0].id
                    new_body.append(ast.With(items=[ast.withitem(context_expr=stmt.value, optional_vars=ast.Name(var, ast.Store()))], body=[mid], type_comment=None))
                    i += 3
                    continue
            new_body.append(stmt); i += 1
        tree.body = new_body
        return ast.fix_missing_locations(tree)

## Tests

In [12]:
import unittest

class TestNoneCompare(unittest.TestCase):
    def setUp(self):
        self.linter = DCCLinter([OpenCloseDetectionRule()],[OpenClosePatcher()])

    def test_detect(self):
        src = "f = open('a.txt','w')\nf.write('x')\nf.close()\n"
        warnings = self.linter.search(src)
        self.assertEqual(len(warnings), 1)

    def test_patch_basic(self):
        src = "f = open('a.txt','w')\nf.write('x')\nf.close()\n"
        out = self.linter.patch(src)
        self.assertIn("with open('a.txt', 'w') as f:", out)
        self.assertNotIn("close()", out)

    def test_skip_when_no_close(self):
        src = "f = open('a.txt','w')\nf.write('x')\n"
        warnings = self.linter.search(src)
        self.assertEqual(len(warnings), 0)
        out = self.linter.patch(src)
        self.assertIn("open('a.txt', 'w')", out)

    # Tests extra
    def test_skip_mismatched_variable(self):
        src = "f = open('a.txt','w')\nf.write('x')\ng.close()\n"
        warnings = self.linter.search(src)
        self.assertEqual(len(warnings), 0)
        out = self.linter.patch(src)
        self.assertIn("g.close()", out)

    def test_patch_read_mode(self):
        src = "f = open('a.txt','r')\nf.read()\nf.close()\n"
        out = self.linter.patch(src)
        self.assertIn("with open('a.txt', 'r') as f:", out)
        self.assertNotIn("close()", out)

    def test_patch_append_mode(self):
        src = "f = open('a.txt','a')\nf.write('y')\nf.close()\n"
        out = self.linter.patch(src)
        self.assertIn("with open('a.txt', 'a') as f:", out)
        self.assertNotIn("close()", out)

In [13]:
runTests(TestNoneCompare)

......
----------------------------------------------------------------------
Ran 6 tests in 0.005s

OK
.....
----------------------------------------------------------------------
Ran 6 tests in 0.005s

OK


## Real World Code

In [14]:
def find_cell(comment):
    """
    Busca la primera celda en el cuaderno actual cuya primera línea
    comience con el comentario dado (por ejemplo, "# Case Study 1").

    Parámetros:
        comment (str): El comentario que se quiere buscar.

    Retorna:
        str | None: El código fuente de la celda encontrada, o None si no existe.
    """
    for cell_source in In:  # `In` contiene el código fuente de todas las celdas ejecutadas
        lines = cell_source.strip().splitlines()
        if lines and lines[0].strip().startswith(comment):  # revisa la primera línea
            return cell_source
    return None

In [17]:
# Case Study 1

# --- Case 1: Simple open/close (should be rewritten) ---
f = open("a.txt", "w")
f.write("hello world")
f.close()

# --- Case 2: Another simple open/close (should be rewritten) ---
g = open("b.txt", "w")
g.write("line 1\n")
g.write("line 2\n")
g.close()

# --- Case 3: Open without close (should NOT be touched) ---
h = open("c.txt", "w")
h.write("oops, no close")

# --- Case 4: Mismatched variable (f opened, g closed → skip) ---
f = open("d.txt", "w")
f.write("wrong close")
g.close()

# --- Case 5: Inside a function (should be rewritten) ---
def save_data(path, text):
    out = open(path, "w")
    out.write(text)
    out.close()

# --- Case 6: Already correct with-statement (should NOT be touched) ---
with open("e.txt", "w") as x:
    x.write("already good")

# Dentro de un if
if True:
    k = open("k.txt", "w")
    k.write("conditional")
    k.close()

# En un bucle
for i in range(2):
    m = open(f"m_{i}.txt", "w")
    m.write(str(i))
    m.close()

# Con try/finally
n = open("n.txt", "w")
try:
    n.write("try-finally")
finally:
    n.close()


In [18]:
linter = DCCLinter([OpenCloseDetectionRule()],[OpenClosePatcher()])

bad_code =  find_cell("# Case Study 1")
warnings = linter.search(bad_code)
print("-= Warnings =-")
for w in warnings:
    print(w)
good_code = linter.patch(bad_code)
print("-= Fixed Code =-")
print(good_code)

-= Warnings =-
CodeWarning(message="Usa 'with open(...) as f:' en lugar de open/close manual", lineno=4, col=0, end_lineno=None, end_col=None)
-= Fixed Code =-
with open('a.txt', 'w') as f:
    f.write('hello world')
g = open('b.txt', 'w')
g.write('line 1\n')
g.write('line 2\n')
g.close()
h = open('c.txt', 'w')
h.write('oops, no close')
f = open('d.txt', 'w')
f.write('wrong close')
g.close()

def save_data(path, text):
    out = open(path, 'w')
    out.write(text)
    out.close()
with open('e.txt', 'w') as x:
    x.write('already good')
if True:
    k = open('k.txt', 'w')
    k.write('conditional')
    k.close()
for i in range(2):
    m = open(f'm_{i}.txt', 'w')
    m.write(str(i))
    m.close()
n = open('n.txt', 'w')
try:
    n.write('try-finally')
finally:
    n.close()
